# Get training data and upload them to S3

In [14]:
%%bash
curl -O -L https://github.com/vpavlin/odh-tensorflow-jobs/raw/master/training/num-dataset.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   162  100   162    0     0     26      0  0:00:06  0:00:06 --:--:--    35
100 4959k  100 4959k    0     0   419k      0  0:00:11  0:00:11 --:--:-- 1169k


In [2]:
my_bucket="ODH-TENSORFLOW-JOBS-DEMO"

import boto3, os
conn = boto3.client(service_name='s3',
    endpoint_url=os.environ['S3_ENDPOINT_URL'])

buckets = [ b['Name'] for b in conn.list_buckets()['Buckets']]
if my_bucket not in buckets:
    resp = conn.create_bucket(Bucket=my_bucket)
    if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
        raise Exception("Could not create bucket:(")
        
    print("Bucket %s created" % my_bucket)
else:
    print("Bucket %s exists" % my_bucket)
    
key = "/input-data/num-dataset.tar.gz"
conn.upload_file(Bucket=my_bucket, Key=key, Filename="num-dataset.tar.gz")
conn.list_objects(Bucket=my_bucket)

Bucket ODH-TENSORFLOW-JOBS-DEMO created


{'ResponseMetadata': {'RequestId': 'tx000000000000000000004-005c863b76-101a-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'transfer-encoding': 'chunked',
   'x-amz-request-id': 'tx000000000000000000004-005c863b76-101a-default',
   'content-type': 'application/xml',
   'date': 'Mon, 11 Mar 2019 10:41:58 GMT'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': '/input-data/num-dataset.tar.gz',
   'LastModified': datetime.datetime(2019, 3, 11, 10, 41, 58, 11000, tzinfo=tzlocal()),
   'ETag': '"938fb559ed093232ca5e985d1f370bf9"',
   'Size': 5078688,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'Ceph demo user', 'ID': 'nano'}}],
 'Name': 'ODH-TENSORFLOW-JOBS-DEMO',
 'Prefix': '',
 'MaxKeys': 1000,
 'EncodingType': 'url'}

In [3]:
key = "/output-model/model_out.tgz"
conn.upload_file(Bucket=my_bucket, Key=key, Filename="model_out.tgz")
conn.list_objects(Bucket=my_bucket)

{'ResponseMetadata': {'RequestId': 'tx000000000000000000006-005c863b76-101a-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'transfer-encoding': 'chunked',
   'x-amz-request-id': 'tx000000000000000000006-005c863b76-101a-default',
   'content-type': 'application/xml',
   'date': 'Mon, 11 Mar 2019 10:41:58 GMT'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': '/input-data/num-dataset.tar.gz',
   'LastModified': datetime.datetime(2019, 3, 11, 10, 41, 58, 11000, tzinfo=tzlocal()),
   'ETag': '"938fb559ed093232ca5e985d1f370bf9"',
   'Size': 5078688,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'Ceph demo user', 'ID': 'nano'}},
  {'Key': '/output-model/model_out.tgz',
   'LastModified': datetime.datetime(2019, 3, 11, 10, 41, 58, 137000, tzinfo=tzlocal()),
   'ETag': '"e747057d9f6d124dd12222a588cd8b43"',
   'Size': 1023846,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'Ceph demo user', 'ID': 'nano'}}],
 'Name

In [ ]:
conn.copy_object(Bucket=my_bucket, CopySource="TEST/output-model/model_out.tgz", Key="/output-model/model_out.tgz")

# Install OpenShift client

In [4]:
%%bash
curl -o oc.tar.gz -L https://github.com/openshift/origin/releases/download/v3.11.0/openshift-origin-client-tools-v3.11.0-0cbc58b-linux-64bit.tar.gz
tar xzf oc.tar.gz
cp openshift-origin-client-tools-v3.11.0-0cbc58b-linux-64bit/oc ~/../bin/oc


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   654    0   654    0     0    114      0 --:--:--  0:00:05 --:--:--   154
100 53.8M  100 53.8M    0     0  4662k      0  0:00:11  0:00:11 --:--:-- 11.7M
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.xattr.security.selinux'


# Check the Service Account

In [5]:
%%bash
oc whoami

system:serviceaccount:tf-demo:jupyter-tensorflow


# List templates

You should see templates imported from [odh-tensorflow-jobs](https://github.com/vpavlin/odh-tensorflow-jobs/tree/master/openshift) repository

In [6]:
%%bash
oc get templates

NAME                                    DESCRIPTION                                                                        PARAMETERS    OBJECTS
jupyter-notebook-workspace-tensorflow   Template for deploying Tensorflow enable Jupyter Notebook images with persist...   4 (all set)   7
odh-config                              Template to configure basic components of ODH ML flow                              3 (3 blank)   1
odh-tensorflow-serving                  Template to serve models using tensorflow                                          5 (all set)   2
odh-tensorflow-serving-knative          Template to serve models using tensorflow and knative                              7 (all set)   1
odh-tensorflow-training                 Template to train models using tensorflow                                          9 (all set)   1


# List parameters for Tensorflow serving template

Once the training job is completed, we can deploy the serving endpoint - see the parametrs below.

In [7]:
%%bash
oc process odh-tensorflow-serving-knative --parameters

NAME                   DESCRIPTION                                                                     GENERATOR           VALUE
APP_NAME               Short name of your application (to be used in OpenShift artifact names)                             demo
INPUT_MODEL_LOCATION   Location where the resulting model will be stored                                                   s3://MY-BUCKET/model-out/
MODEL_NAME             Name of the model                                                                                   mnist
MEMORY                 Memory limit to be assigned to the job                                                              1Gi
CPU                    Limit for number of cores to assign to the job                                                      1
SCALE_FACTOR           Number of requests served by a single instance/pod                                                  1
NAMESPACE              Namespace name the service is supposed to run in (probably curren

# Deploy Tensorflow serving endpoint

In [9]:
%%bash
oc process odh-tensorflow-serving-knative \
    -p APP_NAME=from-jupyter \
    -p INPUT_MODEL_LOCATION="s3://ODH-TENSORFLOW-JOBS-DEMO/output-model" \
    -p MODEL_NAME="intect" \
    -p NAMESPACE=tf-demo \
        | oc apply -f -

service.serving.knative.dev/ohd-tensorflow-serving-knative-from-jupyter created


In [56]:
%%bash
oc get kservice

NAME                                          DOMAIN                                                            LATESTCREATED                                       LATESTREADY                                         READY     REASON
ohd-tensorflow-serving-knative-from-jupyter   ohd-tensorflow-serving-knative-from-jupyter.tf-demo.example.com   ohd-tensorflow-serving-knative-from-jupyter-00001   ohd-tensorflow-serving-knative-from-jupyter-00001   True      


In [57]:
%%bash
oc get pods

NAME                                                              READY     STATUS      RESTARTS   AGE
ceph-nano-0                                                       1/1       Running     0          84m
jupyter-tensorflow-1-744ww                                        2/2       Running     0          87m
jupyter-tensorflow-1-deploy                                       0/1       Completed   0          87m
odh-tensorflow-serving-1-build                                    0/1       Completed   0          88m
odh-tensorflow-training-1-build                                   0/1       Completed   0          88m
ohd-tensorflow-serving-knative-from-jupyter-00001-deployme7z2cb   2/2       Running     0          33s


# Calling prediction endpoint

We need to configure the server name - you can see the name of the deployed service above as `service/odh-tensorflow-serving-from-jupyter` - we'll use the part after slash. The model name needs to match the name defined in `MODEL_NAME` parameter above.

You can use [2.png](https://github.com/vpavlin/odh-tensorflow-jobs/blob/master/2.png) to test the prediction.

In [11]:
%%bash
curl -O -L https://github.com/vpavlin/odh-tensorflow-jobs/raw/master/2.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   140  100   140    0     0     23      0  0:00:06  0:00:05  0:00:01    40
100  1867  100  1867    0     0    161      0  0:00:11  0:00:11 --:--:--   457


### Downloaded image 

![2.png](2.png)

In [12]:
import requests
import json
from scipy import misc
import numpy as np



In [13]:
def preprocess_image(image_file: str):
    """Pre-process the input to match the input expected by the serving.
    ":returns: ndarray, preprocessed image
    """
    image = misc.imread(image_file, mode='L')
    # resize the image to desired format
    image = misc.imresize(image, size=[32, 32])
    # normalize the image
    image = image.astype(dtype=np.float32) / 255
    # expand the dimension
    image = np.expand_dims(image, axis=-1)

    return image.tolist()

In [14]:
# Wrap bitstring in JSON
instance = []
for i in ["2.png", "4.png", "1.png", "9.png"]:
    instance.append(preprocess_image(i))
    
data = json.dumps({"signature_name": "prediction", "instances": instance})


/opt/app-root/lib/python3.6/site-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/opt/app-root/lib/python3.6/site-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [53]:
%%bash --out KSERVICE_DOMAIN

oc get kservice ohd-tensorflow-serving-knative-from-jupyter -o jsonpath='{.status.domain}'

In [54]:
INGRESS_GATEWAY="aaa5c9f9743e711e9aa7f0a0bd8df71a-418337139.us-east-1.elb.amazonaws.com"

In [55]:
json_response = requests.post("http://%s/v1/models/intect:predict" % INGRESS_GATEWAY, data=data, headers={"host": KSERVICE_DOMAIN, "content-type":"application/json"})
json_response.content

b'{\n    "predictions": [["2"], ["4"], ["1"], ["9"]\n    ]\n}'